In [ ]:
# This flips all the images horizontally to double the data
# (From: https://udacity-robotics.slack.com/archives/C7A5HT92M/p1507226777000533)
import os
import glob
from scipy import misc
import numpy as np

def flip_and_save_images(img_dir, extension):
  os.chdir(img_dir)
  files = glob.glob("*." + extension)
  for i, file in enumerate(files):
    print(i)
    img = misc.imread(file, flatten=False, mode='RGB')
    flipped_img = np.fliplr(img)
    misc.imsave("flipped" + file, flipped_img)

################
flip_and_save_images(train_mask_directory, "png")
flip_and_save_images(train_images_directory, "jpeg")

In [ ]:
# Adding checkpoints for each epoch, to see how each one did relative to the previous one
# (From: https://udacity-robotics.slack.com/archives/C7A5HT92M/p1507151193000223)
tensorBoard_cb = keras.callbacks.TensorBoard(log_dir='./logs')
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, period=1)
callbacks = [tensorBoard_cb, checkpoint_cb]

In [ ]:
# Code to delete raw images that do NOT contain the hero:
# (From: https://udacity-robotics.slack.com/archives/C7A5HT92M/p1507092634000032)
import glob
import os
import shutil
import sys
import argparse

import numpy as np
from scipy import misc


def get_cam3_files(files):
    is_cam3 = lambda x: x.find('cam3_') != -1
    return sorted(list(filter(is_cam3, files)))


def get_cam3_file_id(filename, base_path):
    return filename.replace(base_path,'').replace('/','').replace('cam3_','').replace('.png','')


def delete_all_cam_files(id, path):
    c1 = path+'/'+'cam1_'+id+'.png'
    c2 = path+'/'+'cam2_'+id+'.png'
    c3 = path+'/'+'cam3_'+id+'.png'
    c4 = path+'/'+'cam4_'+id+'.png'
    delete_file(c1)
    delete_file(c2)
    delete_file(c3)
    delete_file(c4)


def delete_file(filename):
    try:
        os.remove(filename)
    except OSError:
        pass

def contains_hero(filename):
    # Commenting next 2 lines based on post: https://udacity-robotics.slack.com/archives/C7A5HT92M/p1507092955000027?thread_ts=1507092634.000032&cid=C7A5HT92M
    #    s = np.sum(misc.imread(filename)[:,:,0])
    #    return s < 16711680
    # Instead, we use this:
    return misc.imread(filename)[:,:,0].max() == 255 # (From: https://udacity-robotics.slack.com/archives/C7A5HT92M/p1507092955000027?thread_ts=1507092634.000032&cid=C7A5HT92M)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument('path',
                        help='The image path to filter')

    args = parser.parse_args()

    base_path = args.path
    files = glob.glob(os.path.join(base_path, '*.png'))
    cam3 = get_cam3_files(files)
    for f in cam3:
        if(not contains_hero(f)):
            id = get_cam3_file_id(f, base_path)
            delete_all_cam_files(id, base_path)

In [ ]:
# To determine % of files that contain teh hero:
# (From: https://udacity-robotics.slack.com/files/U4UKR0C5Q/F7DTF3D1C/Script_to_see_what___of_training_image_masks_contain_the_hero.py)
#By tokyo_adam 4-10-17 
import cv2
import numpy as np
import glob, os

#set to the directory where your masks are saved
img_dir = "../data/train/masks/"

total_files = 0
total_hero = 0

os.chdir(img_dir)
for file in glob.glob("*.png"):
	total_files +=1

	img = cv2.imread(file)
	blue = img[:,:,0]

	if np.any(blue == 255):
		total_hero += 1

percent_hero = 100. * total_hero / total_files

print (percent_hero, "percent of files contain the hero")

# Simulator Instructions

During data collection, when does the hero and crowd spawn? Here are the steps i follow:
* mark points for quad patrol as well as hero movement using `P`, `O` and `I`. 
* Switch from local control to patrol/follow mode using `H`.
* Press `M` to spawn the hero and crowd ? This step is actually not very clear from the instructions.

Last point was:
https://udacity-robotics.slack.com/archives/C7A5HT92M/p1507278812000121

In [1]:
# Logging callback for Tensorboard
# (From: https://udacity-robotics.slack.com/archives/C7A5HT92M/p1507762674000185?thread_ts=1507760279.000071&cid=C7A5HT92M)
# 
#logger_cb = plotting_tools.LoggerPlotter()
logger_cb = keras.callbacks.TensorBoard(log_dir='./logs/model1c', histogram_freq=1, batch_size=32, \
  write_graph=True, write_grads=False, write_images=False, \
  embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
save_cb = keras.callbacks.ModelCheckpoint(filepath='./saved_chkpts/model1c/weights-e{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1, period=5)

callbacks = [logger_cb, save_cb]

NameError: name 'keras' is not defined

Using the base dataset, after every run i see that my `iou3` (when hero is far) is very low which is bringing down the final score. I think this is because of less images where hero is far away in the total dataset. I am working on adding more images to the dataset to balance out the classes.

For me the encoder block did matter. Having 2 convolution layers that do not change the size of the layer, before the third convolution that also resizes the layer by using a stride of 2.

I also used 4-5 encoding blocks and the same number of decoding (although not sure that it actually mattered)

Now 33% is very low and I cannot explain it.

Use a lot of epochs.. 100 for example... and do not use all the data for each epoch. The problem is that the dataset is not balanced , so you will probably not get a good result (there are too few images with the target far away)

Here are the counts from training data
```no hero = 2576
hero very far = 521
hero not too close = 621
hero close = 413```

These are calculated based on the number of blue pixels in the `mask.png`. the size of the mask is `256x256`
```no hero = 0 pixels
hero very far = 1 to 40 pixels
hero not tool close = 40 to 400 pixels
hero close = > 400 pixels```

I was able to get 41.6% with base data and without changing the kernel size, i used
```lr = 0.001
epoch = 12
batch size = 32
```
I used 4 encoder layers and 4 decoder layers. strides 2 and upsampled by 2. I tried a lot of different variations but 43% is the max i could do. 20 is my max epochs and i seem to saturate before that.

At every step, the code randomly picks `batch_size` number of images to train. so, steps per epoch is basically how many times to you do that in every epoch. training loss is getting updated at the end of every step and validation loss is getting updated at the end of every epoch.

With just the provided data I could achieve ~41%. Could improve a bit more with adding my own data of the hero. I would echo @annnn's suggestion to try some experiments with model depth and hyperparams! Btw in my experience making really deep model actually made the final performance worse (I presume overfitting) so your milage may vary just with that.

In [ ]:
# Saving model:
filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, period=1)

#...and add it to the callbacks list where the plotting function is called. You can tune what is saved in the checkpoint. That way you can select which ever epoch performs best

# Eg:
logger_cb = plotting_tools.LoggerPlotter()
filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, period=1)

callbacks = [logger_cb,checkpoint_cb]

Best thing to increase the score: collecting your own data. Focus on hero far away

The dropout method is usually used in the last few layers. However, it would lead to slow convergence speed and long training time. As mentioned by @chedanix, if you don't train your model with hundreds  of epochs or you don't notice there is overfitting issue for your model, there is no need to use dropout from my experience.

you can add `model.summary()` before `model.fit_generator` to see the entire summary of the architecture

Visualizing throgh keras
https://keras.io/visualization/

You can use Keras to generate a figure of your network:
If you run a cell with `plotting_tools.plot_keras_model(model, 'name_of_fig')` there will be two plots generated in `/data/figures`. I've used the one with _shapes at the end.

Also, run "conda update --all"

Tip
Try to have more conv layers in encoders and decoders. And turn on the image augmentation. Then set steps of each epoch around 20 to 50.

https://udacity-robotics.slack.com/archives/C7A5HT92M/p1511798020000230

https://udacity-robotics.slack.com/archives/C7A5HT92M/p1513618303000105
    Here's the network image: https://udacity-robotics.slack.com/files/U73U6DK55/F8GP5GQ6R/model.png
    https://udacity-robotics.slack.com/files/U73U6DK55/F8GGSKT2P/image.png
    https://udacity-robotics.slack.com/files/U73U6DK55/F8GP7RR9T/image.png


How to check if I'm using the GPU afterall?:
> In a terminal, type nvidia-smi, on the same machine you are training on. That page will show you the gpu utilization.

> Check this thread out as well if you are using AWS https://udacity-robotics.slack.com/archives/C7A5HT92M/p1510501438000047?thread_ts=1510501438.000047&cid=C7A5HT92M

> https://udacity-robotics.slack.com/archives/C7A5HT92M/p1511251327000239
  https://udacity-robotics.slack.com/archives/C7A5HT92M/p1511758359000169
  https://udacity-robotics.slack.com/archives/C7A5HT92M/p1511917591000148
  
For persistent Jupyter session:
https://udacity-robotics.slack.com/archives/C7A5HT92M/p1510929720000073



About encoders and decoders and 1x1:
https://udacity-robotics.slack.com/archives/C7A5HT92M/p1511695064000027
https://udacity-robotics.slack.com/archives/C7A5HT92M/p1511895827000222

In [ ]:
# Frmo: https://udacity-robotics.slack.com/files/U683C0H52/F8JCWV599/-.py
class BatchIteratorSimple(Iterator):
    def __init__(self, data_folder, batch_size, image_shape,
            num_classes=3, training=True, shuffle=True, seed=None, shift_aug=False):

        self.num_classes = num_classes
        self.shift_aug = shift_aug
        self.data_folder = data_folder
        self.batch_size = batch_size
        self.training = training
        self.image_shape = tuple(image_shape)

        im_files = sorted(glob(os.path.join(data_folder, 'images', '*.jpeg')))
        mask_files = sorted(glob(os.path.join(data_folder, 'masks', '*.png')))

        if len(im_files) == 0:
            raise ValueError('No image files found, check your image diractories')

        if len(mask_files) == 0:
            raise ValueError('No mask files found, check your mask directories')

        self.file_tuples = list(zip(im_files, mask_files))
        self.n = len(self.file_tuples)

        super(BatchIteratorSimple, self).__init__(self.n, batch_size, shuffle, seed)

In [ ]:
# How to visualize the NN:
from tensorflow.contrib.keras.python.keras.utils import plot_model
import pydot

#scroll down to the training section and beaeath the existing line...:
model.compile(optimizer=keras.optimizers.Adam(learning_rate), loss='categorical_crossentropy')
#... add this line
plot_model(model, to_file='model.png')

In [ ]:
# Running on local GPU:
https://udacity-robotics.slack.com/archives/C7A5HT92M/p1514928010000521

In [ ]:
You need to install, into your RoboND environment:
* - Keras: * Use
> pip install git+git://github.com/keras-team/keras.git
Anything but this version will not work.

* - Tensorflow GPU: * Use
> pip install tensorflow-gpu==1.3
*Do not remove Tensorflow 1.2.1*

The correct versions of CUDNN and cudatoolkit should install automatically, in this case cuda 8 and CUDNN 6.